In [1]:
import Modules.SQcircuit_extensions as sq_ext
import Modules.figures as figs
import SQcircuit as sq
import numpy as np
import matplotlib.pyplot as plt
import importlib
from scipy.linalg import eigh
import qutip as qt
import sympy as sym
from joblib import Parallel, delayed, cpu_count

%matplotlib ipympl
plt.rcParams['text.usetex'] = False
importlib.reload(sq_ext)
importlib.reload(sq)
importlib.reload(figs)
np.set_printoptions(linewidth=300, formatter={'float': '{:.12f}'.format})

fF = 1e-15
nH = 1e-9
GHz = 1e9

In [2]:
Δ = 0.1
LF = 26.5
CF = 26.9
EJ = 5.4
ω_r = 6.46
LR = 140
CR = 1 / (LR * nH) / (ω_r * 2 * np.pi * GHz) ** 2 / fF
Lq, Lr = sq_ext.LF_LR_eff_to_Lq_Lr(LF=LF, LR=LR, Δ=Δ)
C, Csh, CJ = sq_ext.CF_CR_eff_to_C_CJ_Csh(CF=CF, CR=CR)
C, Csh, CJ 

(8.671159748277237, 11.282210062930691, 11.282210062930691)

In [29]:

Δ_prime = 0.1
LF_prime = 20.4
CF_prime = 22.4
EJ_prime = 9.5
ω_r_prime = 6.274
# LR_prime = 100
LR_prime = 190
CR_prime = 1 / (LR_prime * nH) / (ω_r_prime * 2 * np.pi * GHz) ** 2 / fF
Lq_prime, Lr_prime = sq_ext.LF_LR_eff_to_Lq_Lr(LF=LF_prime, LR=LR_prime, Δ=Δ_prime)
C_prime, Csh_prime, CJ_prime = sq_ext.CF_CR_eff_to_C_CJ_Csh(CF=CF_prime, CR=CR_prime)
C_prime, Csh_prime, CJ_prime 

(6.773726062354115, 9.50656848441147, 9.50656848441147)

In [30]:
nmax_r = 10
nmax_f = 10
CC = 10
only_inner = True
compensate_extra_cap = False
only_renormalization = False
n_eig = 5
φ_ext = 0.5

In [31]:
C_mat = sq_ext.C_mat_qubit_C_qubit(CC, CR, CF, CR_prime, CF_prime, only_inner, compensate_extra_cap, only_renormalization)
C_mat

array([[29.400000000000, -2.500000000000, 2.500000000000, 2.500000000000],
       [-2.500000000000, 6.835579874139, -2.500000000000, -2.500000000000],
       [2.500000000000, -2.500000000000, 24.900000000000, 2.500000000000],
       [2.500000000000, -2.500000000000, 2.500000000000, 5.886863031177]])

In [32]:
C_inv = np.linalg.inv(C_mat)
C_inv

array([[0.035803356055, 0.008508601662, -0.001646862595, -0.010892002949],
       [0.008508601662, 0.177858242191, 0.010217918960, 0.067579167685],
       [-0.001646862595, 0.010217918960, 0.042665151616, -0.013080128541],
       [-0.010892002949, 0.067579167685, -0.013080128541, 0.208749250905]])

In [33]:
CF_tilde = C_inv[0, 0] ** -1
CR_tilde = C_inv[1, 1] ** -1
CF_prime_tilde = C_inv[2, 2] ** -1
CR_prime_tilde = C_inv[3, 3] ** -1
fluxonium       = sq_ext.sq_fluxonium(C_F_eff=CF_tilde,       L_F_eff=LF,       Δ=Δ,       EJ=EJ,       nmax_f=nmax_f, φ_ext=φ_ext)
resonator       = sq_ext.sq_resonator(C_R_eff=CR_tilde,       L_R_eff=LR,       Δ=Δ,       EJ=EJ,       nmax_r=nmax_r)
fluxonium_prime = sq_ext.sq_fluxonium(C_F_eff=CF_prime_tilde, L_F_eff=LF_prime, Δ=Δ_prime, EJ=EJ_prime, nmax_f=nmax_f, φ_ext=φ_ext)
resonator_prime = sq_ext.sq_resonator(C_R_eff=CR_prime_tilde, L_R_eff=LR_prime, Δ=Δ_prime, EJ=EJ_prime, nmax_r=nmax_r)
fluxonium      .diag(2); 
resonator      .diag(2);
fluxonium_prime.diag(2);
resonator_prime.diag(2);

In [34]:
# 0=F    1=R    2=Fp    3=Rp
C_FR_inv    = C_inv[0,1] * fF**-1
C_FFp_inv   = C_inv[0,2] * fF**-1
C_FRp_inv   = C_inv[0,3] * fF**-1
C_RRp_inv   = C_inv[1,3] * fF**-1
C_FpRp_inv  = C_inv[2,3] * fF**-1

In [35]:
qF  = np.abs( fluxonium       . charge_op(0,basis='eig')[0,1] )
qR  = np.abs( resonator       . charge_op(0,basis='eig')[0,1] )
qFp = np.abs( fluxonium_prime . charge_op(0,basis='eig')[0,1] )
qRp = np.abs( resonator_prime . charge_op(0,basis='eig')[0,1] )

In [36]:
E_FR   = 0.5 * C_FR_inv   * qF  * qR  / 2 / np.pi / GHz
E_FFp  = 0.5 * C_FFp_inv  * qF  * qFp / 2 / np.pi / GHz
E_FRp  = 0.5 * C_FRp_inv  * qF  * qRp / 2 / np.pi / GHz
E_RRp  = 0.5 * C_RRp_inv  * qR  * qRp / 2 / np.pi / GHz
E_FpRp = 0.5 * C_FpRp_inv * qFp * qFp / 2 / np.pi / GHz

In [37]:
E_FR  , E_FFp , E_FRp , E_RRp , E_FpRp

(0.12041551424938861,
 -0.03610801263348697,
 -0.13721066808212132,
 0.4796537447296824,
 -0.25033125231123254)